In [1]:
import pandas as pd
import math # find nan with logic


# Fixing the formatting of these wonderful people
    1. I had to install LibreOffice just to view the files...
    2. Unnamed 0 is the place where the exam takes place at. DONE
    3. Loop over Unnamed 0 if nan then put the previous DONE
    4. Delete the all NaN apart from location, as those are rows that just specify location. DONE
    5. Finally we will append, first just few columns then next which are female and finally the total, we will see later if we use those. DONE
    6. Check if we can convert data to numeric.
    7. LOOP OVER YEARS
    

In [2]:
seasons = [
    "2023-24",
    "2022-23",
    "2021-22",
    "2020-21",
    "2019-20",
    "2018-19",
    "2017-18",
    "2016-17",
    "2015-16",
    "2014-15",
    "2013-14",
    "2012-13",
    "2011-12",
    "2010-11",
    "2009-10",
    "2008-09",
    "2007-08"
]


In [3]:
#Initialize empty dataframe.
df = pd.DataFrame()

# AGE ONLY A COLUMN NAME IN THE LATEST, ALL OTHERS DO NOT HAVE A COLUMN. FIX

In [4]:

for year in seasons:
    #Data has multiple sheets and is formatted horribly.
    
    df_yearly =    pd.read_excel('dvsa1203_v2.ods',engine='odf',sheet_name=year,skiprows=6)
    
    df_yearly = df_yearly.rename(columns={'Unnamed: 0':'Location'})
    # Drop the totals, we can compute later and it is better as they changed it 2017-2018.
    
    #THEY DO NOT HAVE THE AGE COLUMN IN EARLIER DATASETS JESUS.b
    df_yearly = df_yearly[df_yearly['Age']!="Total"]
    
    name = df_yearly.iloc[0,0]
    
    #Put the true names into the file, nan is a float else it is integeer 
    for i in range(df_yearly.shape[0]):
        #If the location is of type float it means it is a missing value we replace it with the previous name.
        #If location is not a float it is a string, all posterior missing values will be that string, until next.
        if type(df_yearly.iloc[i,0])==float:
            
            df_yearly.iloc[i,0] = name
        else:
            name = df_yearly.iloc[i,0]
            
    
    look_na = df_yearly.columns.to_list()
    look_na.remove('Location')
    
    
    # Drop only the rows with only the name of the place and nothing else.
    df_yearly = df_yearly.dropna(how="all",subset=look_na)
    df_yearly = df_yearly.reset_index(drop=True)
    
    #REFORMAT DATA
    
    #Baseline, male
    men = df_yearly.iloc[:,[0,1,2,3,4,]]
    men['gender']=0
    # Females rename the columns
    women = df_yearly.iloc[:,[0,1,5,6,7,]].rename(columns={"Conducted.1":"Conducted","Passes.1":"Passes","Pass rate (%).1":"Pass rate (%)"})
    women['gender']=1
    #Total, added females and males
    
    total = df_yearly.iloc[:,[0,1,8,9,10,]].rename(columns={"Conducted.2":"Conducted","Passes.2":"Passes","Pass rate (%).2":"Pass rate (%)"})
    total['gender']=2
    
    
    df_yearly = pd.concat([men,women,total],axis=0)
    df_yearly['year']=year
    
    df = pd.concat([df_yearly,df],axis=0)
    print(f"{year} completed succesfully")

/var/folders/v_/j4607qws7tq1c9n9g0ynnx340000gp/T/ipykernel_72405/290058877.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  men['gender']=0


2023-24 completed succesfully


KeyError: 'Age'

In [ ]:
df